In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import polars as pl
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("darkgrid")
sns.set_palette("dark")

import os
from tqdm import tqdm

import torch
from sklearn.metrics import mean_squared_error

from pipeline.preprocessing import preprocess
from pipeline.walkforward import WFCVGenerator
from config.config_types import AppConfig
from utils.inference_utils import format_legend_name
from models import create_model
from utils.paths import CONFIG_DIR, SP500COPY_PATH, SP500_PATH, DATA_DIR, VOL_EXPERIMENTS_DIR, PRICE_EXPERIMENTS_DIR
print(SP500_PATH)

DEVICE = "cuda"

In [3]:
cfg_mlp = AppConfig.from_dict(f"{CONFIG_DIR}/vol_mlp.yaml")
cfg_cnn = AppConfig.from_dict(f"{CONFIG_DIR}/vol_cnn.yaml")
cfg_lstm = AppConfig.from_dict(f"{CONFIG_DIR}/vol_lstm.yaml")
cfg_transformer = AppConfig.from_dict(f"{CONFIG_DIR}/vol_transformer.yaml")

wf = WFCVGenerator(cfg_lstm.walkforward, keep_id_col=True)


**Skip if not using OLS**

In [4]:
data = {}



for i, fold_data in enumerate(wf.folds()):
    size = fold_data[4]
    y = fold_data[5]

    data[i] = fold_data
    print(i)
    #if i == fold_num:
    #    break
Xtr, ytr, Xv, yv, Xte, yte, Xtr_val, ytr_val, Xte_merged, yte_merged, id_tr, id_v, id_te, windows_tr, windows_te, windows_v = data[0]


0


In [5]:
X = np.concat([Xtr_val, Xte_merged])
y = np.concat([ytr_val, yte_merged])

In [6]:

from sklearn.linear_model import LinearRegression, Lasso
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. Helper to flatten 3D/2D time-series data for OLS
def prepare_sklearn_input(X):
    X = np.asarray(X)
    if X.ndim == 3: # (N, T, D) -> (N, T*D)
        N, T, D = X.shape
        return X.reshape(N, -1)
    elif X.ndim == 2: # (N, T) -> (N, T)
        return X
    return X

# 2. Prepare the data
# Xtr_val is your training set, Xte is your test set
X_train_lin = prepare_sklearn_input(Xtr_val)
X_test_lin  = prepare_sklearn_input(Xte_merged)

# Ensure targets are the right shape (N,) or (N, 1)
y_train_lin = np.asarray(ytr_val)
y_test_truth = np.asarray(yte_merged)

# 3. Fit OLS
ols_full = LinearRegression()
ols_full.fit(X_train_lin, y_train_lin)

lasso_full = Lasso(fit_intercept=True, alpha=0.05).fit(X_train_lin, y_train_lin)


# 4. Predict
y_pred_te_ols = ols_full.predict(X_test_lin)
y_pred_tr_ols = ols_full.predict(X_train_lin)
y_pred_te_lasso = lasso_full.predict(X_test_lin)
y_pred_tr_lasso = lasso_full.predict(X_train_lin)

# 5. Check Metrics
# Flatten both for safe comparison
mse_ols = mean_squared_error(y_test_truth.ravel(), y_pred_te_ols.ravel()) 
mse_lasso = mean_squared_error(y_test_truth.ravel(), y_pred_te_lasso.ravel()) 
mse_tr_ols  = mean_squared_error(y_train_lin.ravel(), y_pred_tr_ols.ravel())
mse_tr_lasso  = mean_squared_error(y_train_lin.ravel(), y_pred_tr_lasso.ravel())
print(f"OLS Train MSE: {mse_tr_ols:.6f}")
print(f"OLS Test MSE: {mse_ols:.6f}")
print(f"LASSO Test MSE: {mse_lasso:.6f}")

OLS Train MSE: 0.447090
OLS Test MSE: 0.393334
LASSO Test MSE: 0.394691


In [7]:
df_results = pd.read_parquet("df_preds_vol.parquet")
df_results

,permno,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,ols,pred_cnn_100_muon,pred_cnn_100_adam,pred_cnn_100_sgd,pred_lstm_100_muon,pred_lstm_100_adam,pred_lstm_100_sgd,pred_mlp_100_muon,pred_mlp_100_adam,pred_mlp_100_sgd
0,10104,3.022142,3.038342,2.993999,2.888102,2.288038,2.398543,1.626532,2.023018,1.835744,...,2.273912,2.340291,2.304125,1.973967,2.310304,2.284783,2.141823,2.155174,2.277882,2.248590
1,10107,1.966788,1.425903,2.007835,1.607037,1.303735,-0.014046,1.632837,1.133337,2.110379,...,1.060578,1.123310,1.007065,0.930193,0.913475,1.000780,1.108341,0.987555,1.040245,0.992946
2,10138,1.925007,0.744136,1.770868,1.306571,0.597367,0.480277,0.759226,0.764844,2.291407,...,1.196589,1.304105,1.157279,1.416762,1.160508,1.172086,1.292715,1.275568,1.147740,1.191103
3,10145,0.910065,1.414771,0.527388,0.742682,1.236556,0.621537,-0.012326,-0.826894,-1.400075,...,0.628736,0.827664,0.671458,0.838289,0.630944,0.650346,0.629951,0.693746,0.649557,0.608899
4,10147,2.013652,1.852590,2.427796,2.700163,1.775644,1.590639,1.293224,2.191215,2.021293,...,2.113741,2.198706,2.264866,1.808160,2.174373,2.218298,2.046874,1.985762,2.158269,2.170377
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2855196,93096,1.655723,0.875642,0.736836,0.900131,0.497897,-0.026745,1.047596,0.950905,0.261180,...,0.658642,0.683989,0.612032,0.704177,0.599831,0.634295,0.777798,0.616593,0.651973,0.546869
2855197,93132,0.294328,0.919732,0.818286,2.629239,1.113334,0.706085,0.821033,0.402089,0.678040,...,0.379749,0.472100,0.314103,0.445682,0.209755,0.332883,0.447211,0.347395,0.358368,0.309003
2855198,93246,1.709164,2.856059,0.733962,1.034886,1.223298,0.163562,0.233318,0.643048,0.523116,...,0.386820,0.456424,0.356586,0.439740,0.292108,0.376982,0.414978,0.371953,0.327885,0.341250
2855199,93429,0.857807,1.782350,-0.016198,1.324178,0.158603,-0.964462,-0.263984,0.357285,0.238989,...,0.097085,0.145902,0.080869,0.145695,-0.074006,0.041800,0.087685,0.069245,0.064742,0.074166


In [8]:
y_pred_ols = np.concatenate([y_pred_tr_ols, y_pred_te_ols])
y_pred_ols
diff = np.isclose(y_pred_ols, df_results["ols"]) - 1
diff.sum()

np.int64(0)

In [9]:
exp_names = {
    "cnn": {
        "muon": "exp_169_cnn_100_muon_icml_3",
        "adam": "exp_037_cnn_100_adam_lr",
        "sgd" : "exp_041_cnn_100_sgd",
    },
    "lstm": {
        "muon": "exp_036_lstm_100_muon_lr",
        "adam": "exp_039_lstm_100_adam_lr",
        "sgd" : "exp_042_lstm_100_sgd",
    },
    "mlp": {
        "muon": "exp_035_mlp_100_muon_lr",
        "adam": "exp_038_mlp_100_adam_lr",
        "sgd" : "exp_043_mlp_100_sgd",
    },
    "transformer": {
        "adam": "exp_179_transformer_100_adam_lr",
        "muon": "exp_180_transformer_100_muon",
        'sgd': 'exp_200_transformer_100_sgd_v2'
    }
}
# ---- CFG mapping (your cfg objects) ----
cfgs = {
    "cnn":  cfg_cnn,
    "lstm": cfg_lstm,
    "mlp":  cfg_mlp,
    "transformer":  cfg_transformer,
}

# ---- Trial (fixed) ----
trial = "trial_search_best"

# ---- Build the 9 experiments ----
experiments = []
for model in ['cnn', 'mlp', 'lstm', 'transformer']:
    for opt in ["adam", 'muon', 'sgd']:
        experiments.append({
            "name": exp_names[model][opt],
            "cfg":  cfgs[model],
            "trial": trial
        })

names = [exp['name'] for exp in experiments]
trials = [exp['trial'] for exp in experiments]
cfgs = [exp['cfg'] for exp in experiments]

fold_num = 0
base_paths = [f"{VOL_EXPERIMENTS_DIR}/{name}/{trial}/" for name, trial in zip(names, trials)]
ckpt_paths = [f"{base}fold_{fold_num:03d}/model_best.pt" for base in base_paths]
checkpoints = [torch.load(cp, map_location=DEVICE) for cp in ckpt_paths]


In [10]:
experiments

[{'name': 'exp_037_cnn_100_adam_lr',
  'cfg': AppConfig(model=ModelConfig(name='simplecnn', hparams={'mlp_hidden_sizes': [512, 256], 'mlp_activation': ['relu', 'relu'], 'dropout_rate': 0, 'lstm_hidden_sizes': [256, 256], 'lstm_dropout': 0, 'bidirectional': False, 'readout': 'last', 'use_ln': False, 'conv_channels': [64, 128, 256], 'conv_activation': ['relu', 'relu', 'relu'], 'kernel_size': 8, 'padding': 1, 'pool': 'adaptive_avg', 'pool_k': 16, 'use_bn': False, 'output_activation': 'linear'}, search={}), trainer=TrainerConfig(hparams={'epochs': 50, 'batch_size': 512, 'torch_patience': 10, 'min_delta': '1e-10', 'optuna_patience': 10, 'loss': 'mse', 'metrics': ['mae', 'mse', 'dir_acc', 'qlike'], 'val_every': 1, 'optimizer_type': 'muon', 'initialization': 'exp_041_cnn_100_sgd/trial_search_best/fold_000', 'lr': 2.014513088445603e-05, 'weight_decay': 0.000297856597357391}, search={}), walkforward=WFConfig(target_col='var', lookback=0, ratio_train=3, ratio_val=1, ratio_test=1, step=1257, lags

In [ ]:


# ----------------------------
# 1) Infer input shape
# ----------------------------
def _infer_input_shape(cfg, X_sample, state_dict=None):
    """
    Infer model input_shape from a sample. This is only for create_model(...) construction.
    X_sample can be a single observation or a batch; we infer (T,D) or (C,L) or (F,).
    """
    name = cfg.model.name.lower()
    x = np.asarray(X_sample)
    shape = x.shape
    L = cfg.walkforward.lags

    if name in ["lstm", "transformer"]:
        # model expects (T,D)
        if x.ndim == 1:          # (T,)
            return (shape[0], 1)
        if x.ndim == 2:          # (T,D) or (N,T)
            # if looks like (N,T) batch, treat as univariate (T,1)
            if shape[1] == L and shape[0] != L:
                return (shape[1], 1)
            return (shape[0], shape[1])
        if x.ndim == 3:          # (N,T,D)
            return (shape[1], shape[2])
        raise ValueError(f"Unexpected {name} sample shape: {shape}")

    if name == "simplecnn":
        # model expects (C,L), here C=1
        if x.ndim == 1:          # (L,)
            return (1, shape[0])
        if x.ndim == 2:
            # could be (1,L) single sample OR (N,L) batch
            if shape == (1, L):
                return (1, shape[1])
            if shape[1] == L and shape[0] != 1:
                return (1, shape[1])
            # otherwise (T,D) single sample (will be flattened upstream)
            return (1, shape[0] * shape[1])
        if x.ndim == 3:          # (N,1,L) or (N,T,D)
            # if (N,1,L) => return (1,L)
            if shape[1] == 1 and shape[2] == L:
                return (1, shape[2])
            # if (N,T,D) => flatten D later, still length T
            return (1, shape[1])
        raise ValueError(f"Unexpected CNN sample shape: {shape}")

    if name == "mlp":
        # model expects (F,) where here F=L (your case)
        if x.ndim == 1:
            return (shape[0],)
        if x.ndim == 2:
            # (N,L) batch -> (L,)
            if shape[1] == L and shape[0] != L:
                return (shape[1],)
            # (T,D) single sample -> flatten
            return (shape[0] * shape[1],)
        if x.ndim == 3:
            return (shape[1] * shape[2],)
        raise ValueError(f"Unexpected MLP sample shape: {shape}")

    raise ValueError(f"Unknown model {cfg.model.name}")


# ----------------------------
# 2) Block builder that NEVER collapses dims accidentally
# ----------------------------
def _make_block(cfg, X, start, end):
    """
    Build a numeric numpy block with the exact shape expected *before* _prepare_batch:
      - MLP:          (B, L)
      - simplecnn:    (B, 1, L)
      - lstm/transf:  (B, L, 1)
    This avoids np.asarray/list weirdness and the 'np.stack still 2D' issue.
    """
    name = cfg.model.name.lower()
    L = cfg.walkforward.lags
    xs = [np.asarray(X[i]) for i in range(start, end)]

    # stack first (keeps per-sample ndim), then enforce dims
    x_block = np.stack(xs, axis=0)

    if name == "mlp":
        # expected single sample: (L,)
        # allow samples shaped (1,L) too
        if x_block.ndim == 3 and x_block.shape[1:] == (1, L):
            x_block = x_block[:, 0, :]  # (B,L)
        if x_block.ndim != 2 or x_block.shape[1] != L:
            raise ValueError(f"MLP block expected (B,{L}), got {x_block.shape}")
        return x_block

    if name == "simplecnn":
        # expected single sample: (1,L)
        # If samples are (L,), stacked gives (B,L) -> add channel dim
        if x_block.ndim == 2 and x_block.shape[1] == L:
            x_block = x_block[:, None, :]  # (B,1,L)
        # If samples are (1,L), stacked gives (B,1,L) already
        if x_block.ndim != 3 or x_block.shape[1:] != (1, L):
            raise ValueError(f"CNN block expected (B,1,{L}), got {x_block.shape}")
        return x_block

    if name in ["lstm", "transformer"]:
        # expected single sample: (L,1)
        # If samples are (L,), stacked gives (B,L) -> add feature dim
        if x_block.ndim == 2 and x_block.shape[1] == L:
            x_block = x_block[:, :, None]  # (B,L,1)
        if x_block.ndim != 3 or x_block.shape[1:] != (L, 1):
            raise ValueError(f"{name} block expected (B,{L},1), got {x_block.shape}")
        return x_block

    raise ValueError(f"Unknown model name: {name}")


# ----------------------------
# 3) Prepare batch tensor (accepts single sample OR already-shaped block)
# ----------------------------
def _prepare_batch(cfg, x):
    """
    Accepts:
      - single sample in canonical shape:
          MLP: (L,)
          CNN: (1,L)
          LSTM/Trans: (L,1)
      - block in canonical shape:
          MLP: (B,L)
          CNN: (B,1,L)
          LSTM/Trans: (B,L,1)
    Returns torch tensor with batch dim:
          MLP: (B,L)
          CNN: (B,1,L)
          LSTM/Trans: (B,L,1)
    """
    name = cfg.model.name.lower()
    L = cfg.walkforward.lags
    x = np.asarray(x)

    if name == "mlp":
        if x.ndim == 1:
            if x.shape != (L,):
                raise ValueError(f"MLP single expected ({L},), got {x.shape}")
            return torch.as_tensor(x, dtype=torch.float32).unsqueeze(0)
        if x.ndim == 2:
            if x.shape[1] != L:
                raise ValueError(f"MLP batch expected (B,{L}), got {x.shape}")
            return torch.as_tensor(x, dtype=torch.float32)
        raise ValueError(f"MLP unsupported ndim {x.ndim}")

    if name == "simplecnn":
        if x.ndim == 2:
            if x.shape != (1, L):
                raise ValueError(f"CNN single expected (1,{L}), got {x.shape}")
            return torch.as_tensor(x, dtype=torch.float32).unsqueeze(0)  # (1,1,L)
        if x.ndim == 3:
            if x.shape[1:] != (1, L):
                raise ValueError(f"CNN batch expected (B,1,{L}), got {x.shape}")
            return torch.as_tensor(x, dtype=torch.float32)  # (B,1,L)
        raise ValueError(f"CNN unsupported ndim {x.ndim}")

    if name in ["lstm", "transformer"]:
        if x.ndim == 2:
            if x.shape != (L, 1):
                raise ValueError(f"{name} single expected ({L},1), got {x.shape}")
            return torch.as_tensor(x, dtype=torch.float32).unsqueeze(0)  # (1,L,1)
        if x.ndim == 3:
            if x.shape[1:] != (L, 1):
                raise ValueError(f"{name} batch expected (B,{L},1), got {x.shape}")
            return torch.as_tensor(x, dtype=torch.float32)  # (B,L,1)
        raise ValueError(f"{name} unsupported ndim {x.ndim}")

    raise ValueError(f"Unknown model name: {name}")


# ----------------------------
# 4) Build model from checkpoint
# ----------------------------
def build_model_from_checkpoint(cfg, checkpoint, X_example, device="cuda"):
    state_dict = {k.replace("_orig_mod.", ""): v for k, v in checkpoint["model_state"].items()}
    input_shape = _infer_input_shape(cfg, np.asarray(X_example), state_dict)
    output_shape = cfg.walkforward.lookback + 1 if cfg.walkforward.lookback is not None else 1

    model = create_model(cfg.model, input_shape, output_shape)
    model.load_state_dict(state_dict, strict=True)
    model.to(device).eval()
    return model


# ----------------------------
# 5) Inference
# ----------------------------
@torch.inference_mode()
def infer_sequential(
    model,
    cfg,
    X,
    y,
    limit=None,
    autoregressive=False,
    device="cuda",
    block_size=65536,
):
    limit = len(X) if limit is None else min(limit, len(X))

    # --- AR: strictly one-by-one ---
    if autoregressive:
        preds = []
        x_ar = np.asarray(X[0]).copy()

        for _ in tqdm(range(limit), desc="AR inference"):
            xb = _prepare_batch(cfg, x_ar).to(device)
            pb = model(xb).detach().cpu().numpy().reshape(-1)  # (out_dim,)
            preds.append(pb)

            # assumes scalar output
            next_val = pb[0]

            # Roll forward depending on model expected single-sample shape
            name = cfg.model.name.lower()
            L = cfg.walkforward.lags

            if name == "mlp":
                # x_ar: (L,)
                x_ar = np.concatenate([x_ar[1:], [next_val]])
            elif name == "simplecnn":
                # x_ar: (1,L) or (L,) -> keep canonical (1,L)
                if x_ar.ndim == 1:
                    x_ar = np.concatenate([x_ar[1:], [next_val]])
                    x_ar = x_ar[None, :]
                else:
                    x_ar = np.concatenate([x_ar[:, 1:], [[next_val]]], axis=1)
            elif name in ["lstm", "transformer"]:
                # x_ar: (L,1) or (L,) -> keep canonical (L,1)
                if x_ar.ndim == 1:
                    x_ar = np.concatenate([x_ar[1:], [next_val]])[:, None]
                else:
                    x_ar = np.concatenate([x_ar[1:, :], [[next_val]]], axis=0)
            else:
                raise ValueError(f"Unknown model name: {name}")

        preds = np.vstack(preds)

    # --- non-AR: block-batched ---
    else:
        preds_blocks = []
        for start in tqdm(range(0, limit, block_size), desc="Block inference"):
            end = min(start + block_size, limit)

            # critical: build canonical shaped block for this model
            x_block = _make_block(cfg, X, start, end)
            xb = _prepare_batch(cfg, x_block).to(device)

            pb = model(xb).detach().cpu().numpy()
            if pb.ndim == 1:
                pb = pb.reshape(-1, 1)

            preds_blocks.append(pb)

        preds = np.vstack(preds_blocks)

    # --- y_true ---
    y_true = np.asarray(y[:limit])
    if y_true.ndim == 1:
        y_true = y_true.reshape(-1, 1)

    return y_true, preds



In [12]:
def run_all_experiments_sequential(
    experiments,
    checkpoints,
    X,
    y,
    limit=None,
    autoregressive=False,
    device=DEVICE,
):
    """
    For each experiment:
      - build model from checkpoint
      - run infer_sequential
    Returns:
      results: dict[clean_name] = {"y_true": ..., "y_pred": ..., "cfg": cfg}
    """
    results = {}

    for exp, ckpt in zip(experiments, checkpoints):
        cfg = exp["cfg"]
        raw_name = exp["name"]
        legend_name = format_legend_name(raw_name)
        print(legend_name)

        # Build model using one sample as shape example
        model = build_model_from_checkpoint(cfg, ckpt, X[0])

        # Run sequential inference
        y_true, y_pred = infer_sequential(
            model,
            cfg,
            X,
            y,
            limit=limit,
            autoregressive=autoregressive,
            device=device,
        )

        results[legend_name] = {
            "y_true": y_true,
            "y_pred": y_pred,
            "cfg": cfg,
        }

    return results




In [13]:

# --- 1. To get just a list of names ---
clean_names = [format_legend_name(n) for n in names]
clean_names = [f"{n} {i}" if n in clean_names[:i] else n for i, n in enumerate(clean_names)]



results_seq = run_all_experiments_sequential(
    experiments=experiments,
    checkpoints=checkpoints,
    X=X,
    y=y,
    autoregressive=False,   # or True if you want AR mode
    device=DEVICE,
)

for model_name, result in results_seq.items():
    y_true_seq = result["y_true"]
    y_pred_seq = result["y_pred"]



CNN 100 ADAM LR


Block inference:   0%|          | 0/44 [00:00<?, ?it/s]

Block inference: 100%|██████████| 44/44 [00:05<00:00,  7.57it/s]


CNN 100 MUON ICML


Block inference: 100%|██████████| 44/44 [00:04<00:00,  9.22it/s]


...

Block inference: 100%|██████████| 44/44 [00:15<00:00,  2.77it/s]


TRANSFORMER 100 MUON


Block inference: 100%|██████████| 44/44 [00:15<00:00,  2.78it/s]


TRANSFORMER 100 SGD V2


Block inference: 100%|██████████| 44/44 [00:15<00:00,  2.77it/s]


In [14]:
for k, v in results_seq.items():
    v = np.array(v['y_pred']).reshape(-1)
    k = k.lower().replace(' lr', '').replace(' v2', '').replace(' icml', '').replace(' ', '_')
    k = f'pred_{k}'
    print(k)
    try:
        b = df_results[k]
    except KeyError:
        continue
    diff = np.abs(b - v)
    print(np.min(diff), np.percentile(diff, [0.01, 0.05, 0.25, 0.5, 0.75, 0.9, 0.99]), np.max(diff))

pred_cnn_100_adam
0.0 [0.00000000e+00 0.00000000e+00 4.47034836e-08 5.96046448e-08
 1.19209290e-07 1.78813934e-07 2.23517418e-07] 0.0034527779
pred_cnn_100_muon
5.9604645e-08 [1.94025040e-05 9.47713852e-05 4.75924462e-04 9.58442688e-04
 1.43480301e-03 1.72012448e-03 1.89404905e-03] 2.2238216
pred_cnn_100_sgd
0.0 [0.00000000e+00 2.98023224e-08 1.78813934e-07 3.57627869e-07
 5.36441803e-07 6.55651093e-07 7.15255737e-07] 0.001978159
pred_mlp_100_adam
0.0 [0. 0. 0. 0. 0. 0. 0.] 1.1920929e-06
pred_mlp_100_muon
0.0 [0. 0. 0. 0. 0. 0. 0.] 1.1920929e-06
pred_mlp_100_sgd
0.0 [0. 0. 0. 0. 0. 0. 0.] 9.536743e-07
pred_lstm_100_adam
0.0 [0. 0. 0. 0. 0. 0. 0.] 0.00015473366
pred_lstm_100_muon
0.0 [0.00000000e+00 0.00000000e+00 1.19209290e-07 2.38418579e-07
 3.57627869e-07 4.17232513e-07 4.76837158e-07] 0.00036227703
pred_lstm_100_sgd
0.0 [0. 0. 0. 0. 0. 0. 0.] 0.00011187792
pred_transformer_100_adam
pred_transformer_100_muon
pred_transformer_100_sgd


In [15]:
for k, v in results_seq.items():
    y = np.array(v['y_pred']).reshape(-1)
    k = k.lower().replace(' lr', '').replace(' v2', '').replace(' icml', '').replace(' ', '_')
    k = f'pred_{k}'
    print(k)
    
    df_results[k] = y

pred_cnn_100_adam
pred_cnn_100_muon
pred_cnn_100_sgd
pred_mlp_100_adam
pred_mlp_100_muon
pred_mlp_100_sgd
pred_lstm_100_adam
pred_lstm_100_muon
pred_lstm_100_sgd
pred_transformer_100_adam
pred_transformer_100_muon
pred_transformer_100_sgd


In [16]:
df_results

,permno,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,pred_cnn_100_sgd,pred_lstm_100_muon,pred_lstm_100_adam,pred_lstm_100_sgd,pred_mlp_100_muon,pred_mlp_100_adam,pred_mlp_100_sgd,pred_transformer_100_adam,pred_transformer_100_muon,pred_transformer_100_sgd
0,10104,3.022142,3.038342,2.993999,2.888102,2.288038,2.398543,1.626532,2.023018,1.835744,...,1.972734,2.310290,2.284783,2.141825,2.155174,2.277882,2.248590,2.510397,2.379873,2.396359
1,10107,1.966788,1.425903,2.007835,1.607037,1.303735,-0.014046,1.632837,1.133337,2.110379,...,0.929780,0.913423,1.000780,1.108342,0.987554,1.040244,0.992947,0.999340,0.940874,1.033979
2,10138,1.925007,0.744136,1.770868,1.306571,0.597367,0.480277,0.759226,0.764844,2.291407,...,1.415974,1.160521,1.172086,1.292715,1.275568,1.147740,1.191103,1.222052,1.179639,1.146949
3,10145,0.910065,1.414771,0.527388,0.742682,1.236556,0.621537,-0.012326,-0.826894,-1.400075,...,0.837754,0.630940,0.650346,0.629951,0.693746,0.649557,0.608899,0.690371,0.603783,0.644902
4,10147,2.013652,1.852590,2.427796,2.700163,1.775644,1.590639,1.293224,2.191215,2.021293,...,1.807075,2.174467,2.218299,2.046875,1.985762,2.158269,2.170377,2.469499,2.337210,2.362807
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2855196,93096,1.655723,0.875642,0.736836,0.900131,0.497897,-0.026745,1.047596,0.950905,0.261180,...,0.704177,0.599940,0.634295,0.777799,0.616593,0.651973,0.546869,0.637959,0.612851,0.652261
2855197,93132,0.294328,0.919732,0.818286,2.629239,1.113334,0.706085,0.821033,0.402089,0.678040,...,0.445683,0.209783,0.332883,0.447211,0.347395,0.358368,0.309003,0.401731,0.341509,0.330051
2855198,93246,1.709164,2.856059,0.733962,1.034886,1.223298,0.163562,0.233318,0.643048,0.523116,...,0.439741,0.292065,0.376981,0.414978,0.371953,0.327885,0.341250,0.342363,0.313337,0.347904
2855199,93429,0.857807,1.782350,-0.016198,1.324178,0.158603,-0.964462,-0.263984,0.357285,0.238989,...,0.145695,-0.073997,0.041805,0.087685,0.069245,0.064742,0.074166,0.050492,0.029335,-0.000450


In [34]:
import ast
#lags = [ast.literal_eval(df_results["window"].iloc[i]) for i in range(df_results.shape[0])]
windows = [int(df_results['window'].values[i][0]) for i in range(df_results.shape[0])]


In [37]:
df_results['lag'] = windows
df_results


,permno,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,pred_lstm_100_muon,pred_lstm_100_adam,pred_lstm_100_sgd,pred_mlp_100_muon,pred_mlp_100_adam,pred_mlp_100_sgd,pred_transformer_100_adam,pred_transformer_100_muon,pred_transformer_100_sgd,lag
0,10104,3.022142,3.038342,2.993999,2.888102,2.288038,2.398543,1.626532,2.023018,1.835744,...,2.310290,2.284783,2.141825,2.155174,2.277882,2.248590,2.510397,2.379873,2.396359,0
1,10107,1.966788,1.425903,2.007835,1.607037,1.303735,-0.014046,1.632837,1.133337,2.110379,...,0.913423,1.000780,1.108342,0.987554,1.040244,0.992947,0.999340,0.940874,1.033979,0
2,10138,1.925007,0.744136,1.770868,1.306571,0.597367,0.480277,0.759226,0.764844,2.291407,...,1.160521,1.172086,1.292715,1.275568,1.147740,1.191103,1.222052,1.179639,1.146949,0
3,10145,0.910065,1.414771,0.527388,0.742682,1.236556,0.621537,-0.012326,-0.826894,-1.400075,...,0.630940,0.650346,0.629951,0.693746,0.649557,0.608899,0.690371,0.603783,0.644902,0
4,10147,2.013652,1.852590,2.427796,2.700163,1.775644,1.590639,1.293224,2.191215,2.021293,...,2.174467,2.218299,2.046875,1.985762,2.158269,2.170377,2.469499,2.337210,2.362807,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2855196,93096,1.655723,0.875642,0.736836,0.900131,0.497897,-0.026745,1.047596,0.950905,0.261180,...,0.599940,0.634295,0.777799,0.616593,0.651973,0.546869,0.637959,0.612851,0.652261,6184
2855197,93132,0.294328,0.919732,0.818286,2.629239,1.113334,0.706085,0.821033,0.402089,0.678040,...,0.209783,0.332883,0.447211,0.347395,0.358368,0.309003,0.401731,0.341509,0.330051,6184
2855198,93246,1.709164,2.856059,0.733962,1.034886,1.223298,0.163562,0.233318,0.643048,0.523116,...,0.292065,0.376981,0.414978,0.371953,0.327885,0.341250,0.342363,0.313337,0.347904,6184
2855199,93429,0.857807,1.782350,-0.016198,1.324178,0.158603,-0.964462,-0.263984,0.357285,0.238989,...,-0.073997,0.041805,0.087685,0.069245,0.064742,0.074166,0.050492,0.029335,-0.000450,6184


In [18]:
df = pd.read_parquet(SP500_PATH)
df

,permno,date,ret,close,vol,open,high,low,cfacpr
0,10078,2000-01-03,-0.012107,76.5,15270527.0,79.25,79.375,74.5,0.5
1,10078,2000-01-04,-0.062092,71.75,21098710.0,73.4375,75.25,71.5,0.5
2,10078,2000-01-05,0.001742,71.875,30073258.0,70.125,73.5,66.625,0.5
3,10078,2000-01-06,-0.053913,68.0,22741999.0,70.4375,71.875,67.0,0.5
4,10078,2000-01-07,0.056985,71.875,20450260.0,67.1875,72.0,66.75,0.5
...,...,...,...,...,...,...,...,...,...
155298,93436,2024-12-24,0.073572,462.28,59351506.0,435.89999,462.78,435.14001,1.0
155299,93436,2024-12-26,-0.01763,454.13,76392273.0,465.16,465.3299,451.01999,1.0
155300,93436,2024-12-27,-0.049479,431.66,82370345.0,449.51999,450.0,426.5,1.0
155301,93436,2024-12-30,-0.033012,417.41,64705452.0,419.39999,427.0,415.75,1.0


In [ ]:
d_mapping = {}

for i, d in enumerate(np.unique(df['date'])):
    d_mapping[i] = d


df_results['date'] = df_results['lag'].map(d_mapping)
df_results.sort_values(by=['permno', 'date'])


,permno,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,...,pred_lstm_100_adam,pred_lstm_100_sgd,pred_mlp_100_muon,pred_mlp_100_adam,pred_mlp_100_sgd,pred_transformer_100_adam,pred_transformer_100_muon,pred_transformer_100_sgd,lag,date
28089,10078,2.526483,1.963724,1.467432,1.512150,2.044465,2.777617,2.568463,3.457545,2.111343,...,0.519403,0.512215,0.574672,0.538155,0.698205,0.651942,0.604054,0.533246,65,2000-04-05
28536,10078,1.961486,1.466110,1.510708,2.042783,2.775471,2.567286,3.455389,2.110057,1.957647,...,0.719455,0.692659,0.742073,0.744121,0.830880,0.817028,0.809103,0.774431,66,2000-04-06
28984,10078,1.464309,1.509362,2.041102,2.773444,2.565236,3.454004,2.108586,1.956410,1.508148,...,0.646997,0.658405,0.679183,0.681962,0.728754,0.707213,0.701586,0.687228,67,2000-04-07
29431,10078,1.507523,2.039456,2.771435,2.563309,3.451547,2.107517,1.955017,1.507051,1.759150,...,0.561669,0.556677,0.568609,0.536480,0.628903,0.592264,0.604396,0.575059,68,2000-04-10
29878,10078,2.037151,2.769377,2.561394,3.449202,2.105679,1.953983,1.505887,1.757974,1.624384,...,0.462371,0.438211,0.477059,0.444628,0.560566,0.500738,0.507082,0.453734,69,2000-04-11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2853230,93436,0.908745,1.359408,0.859101,2.222960,1.297665,2.861383,1.802444,1.579436,1.432361,...,1.370511,1.496492,1.325204,1.380374,1.308432,1.426067,1.361997,1.453587,6180,2024-07-29
2853723,93436,1.357701,0.858122,2.221198,1.296334,2.859198,1.801446,1.578235,1.431288,0.602104,...,1.893088,1.744725,1.644862,1.861510,1.901139,2.169747,2.046722,2.085916,6181,2024-07-30
2854216,93436,0.856856,2.219451,1.294989,2.857132,1.799749,1.577290,1.430162,0.601291,0.328955,...,1.938290,1.830788,1.684414,1.872242,1.886836,2.147994,2.023381,2.127311,6182,2024-07-31
2854708,93436,2.216987,1.293765,2.855085,1.798181,1.575696,1.429251,0.600588,0.328228,1.319478,...,1.940254,1.866197,1.725731,1.880178,1.933082,2.094851,2.016802,2.078630,6183,2024-08-01


In [43]:
df_results = df_results.drop(columns='lag')
df_results.to_parquet('df_preds_vol_v2.parquet')

In [ ]:
sp500_daily_data = pd.read_parquet(SP500_PATH)
sp500_daily_data["date"] = pd.to_datetime(sp500_daily_data["date"])
sp500_daily_data

In [ ]:
#Merge with vol data
df_pl = pl.read_parquet("df_preds_vol_v2.parquet")
df = df_pl.to_pandas()
df["date"] = pd.to_datetime(df["date"])

In [ ]:
rawq = sp500_daily_data.sort_values(["permno", "date"])
rawq["t_idx"] = rawq.groupby("permno").cumcount()
df["k"] = df["window"].apply(lambda x: int(x[0]))
df["t_idx_target"] = df["k"] + 101
cols_keep = [c for c in df.columns if not c.startswith("feature_")]
df_nofeatures = df[cols_keep]

In [ ]:
merged = df_nofeatures.merge(
    rawq,
    left_on=["permno", "date"],
    right_on=["permno", "date"],
    how="left",
    validate="many_to_one"
)

merged = merged.dropna()
merged.reset_index(drop=True, inplace=True)


In [ ]:
merged

In [ ]:
merged = merged.sort_values(["date", "permno"])
merged.to_parquet("sp500_vol_forecasts_2000_2024.parquet")